Calculate axes for all views

In [1]:
import cv2
import os
import covMatrix
from statistics import mean
import math

Define img folders for different fruit types

In [2]:
sphere_path = "./data/oranges/obj0001"
oblate_path = "./data/tomatoes/obj0001"
prolate_path = "./data/mandarins/obj0001"    # Fixme: Not sure if mandarin is prolate model

fruits_types_dict = {
    "sphere": sphere_path,
    "oblate": oblate_path,
    "prolate": prolate_path
}

Define global vars

In [3]:
selected_fruit = "prolate"  #Type of fruit (sphere, oblate, prolate)
stride = 2  # This is the stride length to downsample the masked image
A = 0  # Principal major axis of spheroid
B = 0  # Principal minor axis of spheroid
a_all_views = []
b_all_views = []

image_files = [f for f in os.listdir(fruits_types_dict[selected_fruit])]  # Create list of file names in that folder

Cycle views and get axes for each one

In [4]:
for image_name in image_files:
    # Read img
    file_path = os.path.join(fruits_types_dict[selected_fruit], image_name)
    img = cv2.imread(file_path)

    # Generate binary mask for img
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_img, 0, 1, cv2.THRESH_BINARY)

    # Downsample image with stride to improve performance in computing covar matrix
    strided_image = mask[::stride, ::stride]

    axes = covMatrix.calculate_projected_axes(strided_image)  # Returns [a, b] where a > b
    a_all_views.append(axes[0])
    b_all_views.append(axes[1])

Make the spheroid model with axes from all views

In [5]:
if selected_fruit == "sphere":
    axes_sum = [(x + y) / 2 for x, y in zip(a_all_views, b_all_views)]
    A = mean(axes_sum)
    B = mean(axes_sum)
elif selected_fruit == "oblate":
    A = mean(a_all_views)
    B = min(b_all_views)
elif selected_fruit == "prolate":
    A = max(a_all_views)
    B = mean(b_all_views)
else:
    print("Selected fruit value is not a valid type")

In [6]:
print("For fruit ", selected_fruit, "\n\t", "The spheroid major axis (A): ", A, "\n\t", "The spheroid minor axis (B): ", B)

For fruit  prolate 
	 The spheroid major axis (A):  99.66756611121451 
	 The spheroid minor axis (B):  88.72767054312774
